<span style="color:red">Abgegeben von (Name, Vorname):</span> 
Musterfrau, Marina

# WordNet II: Distanz- und Ähnlichkeitsmaße 

Für dieses Notebook gibt es eine begeleitendes Skript auf Ilias: `skript-wordnet.pdf`

Die Basiselemente von [WordNet](https://wordnet.princeton.edu) wurde in der letzten Sitzung bereits vorgestellt. In dieser Sitzung beschäftigen wir uns mit dem (indirekten) Verhältnis zwischen den Synsets in WordNet, wie wir damit die konzeptuelle Ähnlichkeit bemessen und schließlich ein Wort in einem Satz disambiguieren können.

Wir importieren wie gewohnt das WordNet-Modul von NLTK. 

In [ ]:
from nltk.corpus import wordnet as wn

Immer griffbereit:
- Website: https://www.nltk.org/
- Buch: https://www.nltk.org/book/
- Module: https://www.nltk.org/py-modindex.html
- Beispiele: http://www.nltk.org/howto/

## Distanzmaße: Minimale Pfadlänge

Die Synsets in WordNet sind über Relationskanten (Hyponomie, Hyperonymie, Meronymie, Antonymie, ...) verbunden, die als Sequenz einen sogenannten **Pfad** bilden. Da wir es hier mit einem Graphen zu tun haben, kann es mehrere solcher Pfade geben (im Unterschied zu Bäumen). 

![aus Fellbaum (2006)](Fellbaum(2006)-Figure1-part.PNG)

Am interessantesten ist sicherlich der kürzeste Pfad zwischen zwei Synsets: 

$$pathlen(s_1 ,s_2 ) = \text{die Anzahl der Kanten im kürzesten Pfad zwischen } s_1 \text{ und }
s_2$$

NLTK stellt hier die Methode `shortest_path_distance()` für Synset-Objekte zur Verfügung. Aber **Vorsicht**: `shortest_path_distance()` betrachtet nur die Hyponymie-Relation.




In [ ]:
print("Ergebnisse von shortest_path_distance() bei unterschiedlichen Relationstypen:")
print("gemeinsames direktes Hyponym: {}".format(wn.synset('organism.n.01').shortest_path_distance(wn.synset('causal_agent.n.01'))))
print("Hyponymie: {}".format(wn.synset('bank.n.01').shortest_path_distance(wn.synset('waterside.n.01'))))
print("Meronymie: {}".format(wn.synset('water.n.01').shortest_path_distance(wn.synset('oxygen.n.01'))))
print("Antonymie: {}".format(wn.synset('living.n.02').shortest_path_distance(wn.synset('dead.n.01'))))
print("Derivationally related: {}".format(wn.synset('decision.n.01').shortest_path_distance(wn.synset('decide.v.01'))))

### <span style="color:red">Aufgaben I</span>

<span style="color:red">A1:</span> Schreiben Sie eine Funktion `shortest_hyponomy_pathlength_of_lemmas(lemma1,lemma2)`, die für zwei **Lemmaformen** `lemma1` und `lemma2` den kürzesten Hyponomie-Pfad als ganze Zahl und die entsprechenden Synset-Namen ausgibt! Verwenden Sie als Ausgabeformat eine Liste mit drei Elementen:

    shortest_hyponomy_pathlength_of_lemmas('bank','waterside') -->  [1,'bank.n.01','waterside.n.01']



In [ ]:
# Lösung A1



In [ ]:
# Test für A1

["{:10}, {:10}: {}".format(lemma1,lemma2,shortest_hyponomy_pathlength_of_lemmas(lemma1,lemma2))
    for  lemma1,lemma2 
    in [['bank','waterside'],
        ['person','cause'],
        ['person','organism'],
        ['cause','organism']]]


## Ähnlichkeitsmaße

Es gibt eine Reihe von Größen, die die semantische Ähnlichkeit oder semantische Nähe von zwei Synsets bemessen soll. 

### Pfadbasiert

Die Pfadlänge an sich ist ein wichtiger und einfacher Indikator für die semantische Ähnlichkeit oder semantische Nähe von zwei Synsets: Je kleiner die (kleinste) Pfadlänge, desto ähnlicher sind die Synsets und die enthaltenen Lexeme/Lemmata.

$$sim_{path}(s_1,s_2) = \frac{1}{pathlen(s_1,s_2)+1}$$

NLTK stellt dafür die Methode `path_similarity()` zur Verfügung.

### Pfadbasiert + LCS

Eine Variante der pfadbasierten Ähnlichkeit, die die Tiefe der Synsets berücksichtigt ist die **Wu-Palmer Similarity** (`wup-similarity()`). Die Idee ist hier, dass sich die Einbettungstiefe $depth$ des "lowest common subsumer" $lcs$ (niedrigstes gemeinsames Hypernym) positiv auf das Ähnlichkeitsmaß auswirkt. 

$$sim_{wup}(s_1,s_2) = \frac{2 * depth(lcs(s_1,s_2))}{pathlen(s_1,lcs(s_1,s_2))+pathlen(s_2,lcs(s_1,s_2))}$$

### Wahrscheinlichkeiten der Synsets und des LCS

Es gibt aber auch Ähnlichkeitsmaße, die Pfade außer Acht lassen und nur die Wahrscheinlichkeit der Synsets und deren LCS betrachten. Die benötigten Wahrscheinlichkeiten können z.B. mit Wort-Frequenzen in Corpora abgeschätzt werden:

$$P(s) = \frac{\sum_{w \in lemmas(s)} count(w)}{N}$$

Die Wahrscheinlichkeit der Synsets nimmt damit tendenziell ab, je spezifischer sie sind: 

<img src="Jurafsky,Martin(2018)-FigureC.6.PNG" alt="aus Jurafsky & Martin (2018)" style="width: 220px;"/>

Ein bekanntes Ähnlichkeitsmaß ist hier die **Resnik Similarity** (`res-similarity()`), die nur die Wahrscheinlichkeit (oder genauer gesagt den Informationsgehalt) des LCS betrachtet:

$$sim_{Resnik}(s_1,s_2) = -\log P(lcs(s_1,s_2))$$

Weitentwicklungen der Resnik Similarity berücksichtigen auch die Wahrscheinlichkeit der Synsets (und damit indirekt die Pfadlänge zum LCS).

NLTK enthält davon:
- **Lin Similarity** (`lin-similarity()`): $sim_{Lin}(s_1,s_2) = \frac{2 * \log P(lcs(s_1,s_2))}{\log P(s_1) + P(s_2)}$
- **Jiang-Conrath Distance** (`jcn_similarity()`): $sim_{JC}(s_1,s_2) = 2 * \log P(lcs(s_1,s_2)) - (\log P(s_1) + P(s_2))$ 

In [ ]:
# Die wahrscheinlichkeitsbezogenen Maße benötigen zusätzliche Daten, 
# die zuvor importiert werden müssen.  
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

def compare_similarity_measures(s1,s2) :
    print("Synsets: {}, {}".format(s1.name(),s2.name()))
    print("path_sim: {}".format(s1.path_similarity(s2)))
    print("wup_sim: {}".format(s1.wup_similarity(s2)))
    print("res_sim: {}".format(s1.res_similarity(s2,brown_ic)))
    print("lin_sim: {}".format(s1.lin_similarity(s2,brown_ic)))
    print("jcn_sim: {}".format(s1.jcn_similarity(s2,brown_ic)))

compare_similarity_measures(wn.synset('bank.n.1'),wn.synset('sea.n.1'))
print("")
compare_similarity_measures(wn.synset('bank.n.1'),wn.synset('bank.n.2'))
print("")
compare_similarity_measures(wn.synset('organism.n.01'),wn.synset('causal_agent.n.01'))

### <span style="color:red">Aufgaben II</span>

Wir können nun Aufgabe A1 so erweitern, dass $lemmasim$ für zwei Lemmaformen ($l_1, l_2$) und eines der genannten Ähnlichkeitsmaße ($sim$) berechnet wird.

$$lemmasim(l_1,l_2) = \max_{s_1 \in synsets(l_1)\\s_2 \in synsets(l_2)} sim(s_1,s_2)$$

<span style="color:red">A2:</span> Schreiben Sie eine Funktion `lemmasim(l1,l2,sim)`, die bezogen auf ein Ähnlichkeitsmaß `sim` (path, wup, res, lin, jcn) den höchsten Wert und die dazugehörigen Synsets ausgibt!  



In [ ]:
# Lösung A2



In [ ]:
# Tests für A2

["{:10}, {:10}: {}".format(lemma1,lemma2,lemmasim(lemma1,lemma2,"res"))
    for  lemma1,lemma2 
    in [['bank','waterside'],
        ['person','cause'],
        ['person','organism'],
        ['cause','organism']]]

## Exkursion: SemCor

SemCor ist ein mit WordNet-Synsets annotierter Teil des Brown-Corpus (u.a.) und umfasst ca. 240 000 Worttoken.

SemCor ist in NLTK enthalten und als `corpus.reader`-Modul abrufbar: https://www.nltk.org/api/nltk.corpus.reader.html#module-nltk.corpus.reader.semcor 


In [ ]:
from nltk.corpus import semcor

list(map(str, semcor.tagged_chunks(tag='both')[:3]))

## Anwendung: Disambiguierung von *interest*

Wie können wir nun mit den oben behandelten Ähnlichkeitsmaßen ein Token disambiguieren? Und wie unterscheiden sich die Ähnlichkeitsmaße in ihrer "Treffsicherheit"?

Wir werden uns hier auf ein Nomen konzentrieren, nämlich *interest* mit seinen 7 Bedeutungen:

In [ ]:
[[syns.name(),syns.definition()] for syns in wn.synsets('interest',pos='n')]

Warum *interest*? Weil dafür Testdaten aus einem [Senseval-Wettbewerb](https://en.wikipedia.org/wiki/SemEval) in NLTK vorliegen.

### Exkurs: Vorbereitung Senseval-Testdaten

Die Senseval-Testdaten sind Teil von NLTK (https://www.nltk.org/api/nltk.corpus.reader.html#module-nltk.corpus.reader.senseval), müssen aber zunächst vorbereitet werden. 

Es gibt unterschiedliche "Dateien" für unterschiedliche Lemmaformen, die bei dem Senseval-Wettbewerb disambiguiert werden sollten ("target words"):

In [ ]:
from nltk.corpus import senseval
senseval.fileids()

Davon interessiert uns aber nur `interest.pos`:

In [ ]:
from nltk.corpus import senseval
senseval.instances('interest.pos')

Die Attribute dieser Instanzen können wir ganz leicht einzeln ausgeben: `senseval.instances('interest.pos')[0].word` etc.

Wir stellen aber fest, dass die Synset-Namen nicht stimmen (z.B. `interest_6`). Es handelt sich dabei um alte WordNet-Bezeichnungen, die wir mit `SV_SENSE_MAP` übersetzen werden: 

In [ ]:
# Copied from https://stackoverflow.com/a/16391584/6452961

# A map of SENSEVAL senses to WordNet 3.0 senses.
# SENSEVAL-2 uses WordNet 1.7, which is no longer installable on most modern
# machines and is not the version that the NLTK comes with.
# As a consequence, we have to manually map the following
# senses to their equivalent(s).
SV_SENSE_MAP = {
    "HARD1": ["difficult.a.01"],    # not easy, requiring great physical or mental
    "HARD2": ["hard.a.02",          # dispassionate
              "difficult.a.01"],
    "HARD3": ["hard.a.03"],         # resisting weight or pressure
    "interest_1": ["interest.n.01"], # readiness to give attention
    "interest_2": ["interest.n.03"], # quality of causing attention to be given to
    "interest_3": ["pastime.n.01"],  # activity, etc. that one gives attention to
    "interest_4": ["sake.n.01"],     # advantage, advancement or favor
    "interest_5": ["interest.n.05"], # a share in a company or business
    "interest_6": ["interest.n.04"], # money paid for the use of money
    "cord": ["line.n.18"],          # something (as a cord or rope) that is long and thin and flexible
    "formation": ["line.n.01","line.n.03"], # a formation of people or things one beside another
    "text": ["line.n.05"],                 # text consisting of a row of words written across a page or computer screen
    "phone": ["telephone_line.n.02"],   # a telephone connection
    "product": ["line.n.22"],       # a particular kind of product or merchandise
    "division": ["line.n.29"],      # a conceptual separation or distinction
    "SERVE12": ["serve.v.02"],       # do duty or hold offices; serve in a specific function
    "SERVE10": ["serve.v.06"], # provide (usually but not necessarily food)
    "SERVE2": ["serve.v.01"],       # serve a purpose, role, or function
    "SERVE6": ["service.v.01"]      # be used by; as of a utility
}

Jetzt können wir die Umwandlung der Senseval-Daten für *interest* vornehmen:

In [ ]:
from nltk.corpus import senseval
interestGoldData = [[SV_SENSE_MAP[inst.senses[0]][0],inst.position,inst.context] 
                        for inst in senseval.instances('interest.pos')]
interestTestData = [['', inst[1], inst[2]] for inst in interestGoldData]

print(interestGoldData[0])
print(interestTestData[0])

### <span style="color:red">Aufgaben III</span>

Mit den Testdaten in `interestTestData` und den Golddaten in `interestGoldData` können wir verschiedene Ansätze und Ähnlichkeitsmaße für die Disambiguierung von *interest* evaluieren. 

<span style="color:red">A3:</span> Vervollständigen Sie die Funktion `disambiguate_interest`, indem Sie mindestens eine der oben erwähnten Ähnlichkeitsmetriken verwenden!

In [ ]:
def disambiguate_interest(inst) :
    outsyns = inst[0]
    position = inst[1]
    context = inst[2]   # [('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ...]
    
    # Lösung A3
 
    
    return outsyns   

In [ ]:
# Test für A3

sumTrueDisambiguations = 0
sumFalseDisambiguations = 0

for i in range(len(interestTestData)) :
    if disambiguate_interest(interestTestData[i]) == interestGoldData[i][0] :
        sumTrueDisambiguations += 1
    else :
        sumFalseDisambiguations += 1

accuracyDisambiguations = sumTrueDisambiguations /(sumTrueDisambiguations + sumFalseDisambiguations)

print("Accuracy: {}".format(accuracyDisambiguations))